# Quality control

In [2]:
import hail as hl
hl.init(spark_conf={'spark.driver.memory': '10g'}, tmp_dir='/home/olavur/tmp')

2021-09-30 14:47:32 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2021-09-30 14:47:33 WARN  Hail:37 - This Hail JAR was compiled for Spark 2.4.5, running with Spark 2.4.1.
  Compatibility is not guaranteed.
2021-09-30 14:47:33 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 2.4.1
SparkUI available at http://hms-beagle-6676655f87-78dcj:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/fargen-1-exome/notebooks/qc/hail-20210930-1447-0.2.61-3c86d3ba497a.log


In [3]:
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models.scales import LogScale
output_notebook()

Loading BokehJS ...

In [4]:
import pandas as pd

Load variant data.

In [5]:
BASE_DIR = '/home/olavur/experiments/2020-11-13_fargen1_exome_analysis'
mt = hl.read_matrix_table(BASE_DIR + '/data/mt/variants.mt')

In [287]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

[Stage 169:=========================================>            (46 + 14) / 60]

Number of variants: 2400930
Number of samples: 473


[Stage 170:==========================================>           (47 + 13) / 60]

## Variant QC

### Variant counts

In [288]:
def variant_counts(mt):
    # Count number of variants, SNPs and indels. Only first allele in alternate allele list is considered.
    variant_counts_struct = mt.aggregate_rows(hl.struct(
        n_variants = hl.agg.count(),
        snps_fraction = hl.agg.count_where(hl.is_snp(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        insertions_fraction = hl.agg.count_where(hl.is_insertion(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        deletions_fraction = hl.agg.count_where(hl.is_deletion(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        mnp_fraction = hl.agg.count_where(hl.is_mnp(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        complex_fraction = hl.agg.count_where(hl.is_complex(mt.alleles[0], mt.alleles[1])) / hl.agg.count(),
        star_fraction = hl.agg.count_where(hl.is_star(mt.alleles[0], mt.alleles[1])) / hl.agg.count()))
    
    variant_counts_pd = pd.DataFrame(variant_counts_struct.values(), index=variant_counts_struct.keys(), columns=[''])
    return variant_counts_pd

Create a dataset where multi-allelic sites are split.

In [6]:
# NOTE: permit_shuffle is needed when there are duplicate loci, one of which has multiple alternate alleles. Apparently, this dataset has such sites.
split_mt = hl.split_multi_hts(mt, permit_shuffle=True)

In [289]:
variant_counts(split_mt)

[Stage 171:==============================================>     (108 + 12) / 120]

,
n_variants,2.400930e+06
snps_fraction,5.739692e-01
insertions_fraction,2.652214e-01
deletions_fraction,1.608094e-01
mnp_fraction,0.000000e+00
complex_fraction,0.000000e+00
star_fraction,0.000000e+00


### Variant call quality (QUAL)

Plot distribution of QUAL values. Note that a hard cut-off filter has been used on QUAL in the joint genotyping pipeline.

In [290]:
p = hl.plot.histogram(mt.qual, legend='Variant call quality (QUAL)')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 173:====================================================>(118 + 2) / 120]

In [291]:
p = hl.plot.histogram(hl.log10(mt.qual), legend='Variant call quality (log10 of QUAL)')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 175:===================================================> (117 + 3) / 120]

### VQSR filters

We will inspect the VQSR tranches.

**FIXME:** These variants with `filters = None` have `VQSLOD = NaN`, as shown in the Pandas table below. What are these variants? Why have they not been assigned a VQSLOD?

In [292]:
mt = mt.transmute_rows(filters=hl.delimit(mt.filters, ','))

In [293]:
mt.aggregate_rows(hl.agg.counter(mt.filters))

[Stage 176:==========================================>          (96 + 24) / 120]

{'': 154901,
 None: 1200465,
 'VQSRTrancheINDEL99.90to100.00': 61998,
 'VQSRTrancheINDEL99.00to99.90': 98982,
 'VQSRTrancheSNP99.90to100.00': 72641,
 'VQSRTrancheSNP99.00to99.90': 143262,
 'VQSRTrancheINDEL90.00to99.00': 135371,
 'VQSRTrancheSNP90.00to99.00': 112581,
 'VQSRTrancheSNP50.00to75.00': 103264,
 'VQSRTrancheINDEL50.00to75.00': 65385,
 'VQSRTrancheSNP75.00to90.00': 93967,
 'VQSRTrancheINDEL75.00to90.00': 158113}

We will calculate some mean QC statistics for each VQSR tranch.

In [294]:
# Calculate variant statistics.
mt = hl.variant_qc(mt)

# Get rows table.
rows_ht = mt.rows()

# Aggregate.
result = (rows_ht.group_by(rows_ht.filters)
         .aggregate(mean_gq = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.gq_stats.mean), hl.agg.mean(rows_ht.variant_qc.gq_stats.mean)),
                   mean_dp = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.dp_stats.mean), hl.agg.mean(rows_ht.variant_qc.dp_stats.mean)),
                   mean_af = hl.agg.filter(~hl.is_nan(rows_ht.variant_qc.AF[0]), hl.agg.mean(1 - rows_ht.variant_qc.AF[0])),
                   mean_vqslod = hl.agg.filter(~hl.is_nan(rows_ht.info.VQSLOD), hl.agg.mean(rows_ht.info.VQSLOD)),
                   n_variants = hl.agg.count()))

We convert the results to a Pandas dataframe.

In [295]:
vqsr_stats_pd = result.to_pandas()

[Stage 177:==================================================>  (115 + 5) / 120]2021-09-30 13:42:17 Hail: INFO: Ordering unsorted dataset with network shuffle


Below we first print the statistics for the SNPs and then for the indels. The rows are sorted by mean genotype quality. The empty filter row (`filter=''`) corresponds to all unfiltered variants.

In [296]:
vqsr_stats_pd[vqsr_stats_pd.filters.isin(['VQSRTrancheSNP50.00to75.00', 'VQSRTrancheSNP75.00to90.00', 'VQSRTrancheSNP90.00to99.00', 'VQSRTrancheSNP99.00to99.90', 'VQSRTrancheSNP99.90to100.00', '', None])].sort_values('mean_vqslod')

,filters,mean_gq,mean_dp,mean_af,mean_vqslod,n_variants
10,VQSRTrancheSNP99.90to100.00,39.781591,30.639272,0.033365,-48.666990,72641
9,VQSRTrancheSNP99.00to99.90,41.256575,23.596170,0.055139,-2.138951,143262
8,VQSRTrancheSNP90.00to99.00,56.848757,26.033100,0.119880,1.310761,112581
7,VQSRTrancheSNP75.00to90.00,61.561589,27.342556,0.124890,2.819455,93967
6,VQSRTrancheSNP50.00to75.00,68.090764,30.222961,0.183544,4.374470,103264
0,,72.879413,34.288755,0.263123,6.006016,154901
11,None,52.422394,26.054926,0.094518,NaN,1200465


In [297]:
vqsr_stats_pd[vqsr_stats_pd.filters.isin(['VQSRTrancheINDEL50.00to75.00', 'VQSRTrancheINDEL75.00to90.00', 'VQSRTrancheINDEL90.00to99.00', 'VQSRTrancheINDEL99.00to99.90', 'VQSRTrancheINDEL99.90to100.00', '', None])].sort_values('mean_vqslod')

,filters,mean_gq,mean_dp,mean_af,mean_vqslod,n_variants
5,VQSRTrancheINDEL99.90to100.00,64.825060,37.630893,0.016393,-14.773025,61998
4,VQSRTrancheINDEL99.00to99.90,40.318224,21.808810,0.016740,-3.462648,98982
3,VQSRTrancheINDEL90.00to99.00,33.259442,15.810880,0.028766,1.352714,135371
2,VQSRTrancheINDEL75.00to90.00,47.140238,21.853790,0.041205,3.251756,158113
1,VQSRTrancheINDEL50.00to75.00,55.977036,25.266741,0.078265,4.134389,65385
0,,72.879413,34.288755,0.263123,6.006016,154901
11,None,52.422394,26.054926,0.094518,NaN,1200465


## Genotype QC

In [298]:
p = hl.plot.histogram(mt.GQ, range=(0, 100))
p.xaxis.axis_label = 'Genotype quality'
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 182:===================================================> (117 + 3) / 120]

### Allelic balance

Note that since we've split the table, all rows are diallelic.

We compute the allelic balance as $AB = \frac{AD[1]}{DP}$. Note that DP is equivalent to $AD[0] + AD[1]$ for a diallelic site.

**NOTE:** It is important that we use the split dataset here.

In [299]:
split_mt = split_mt.annotate_entries(AB = split_mt.AD[1] / split_mt.DP)

In [300]:
hets_mt = split_mt.filter_entries(split_mt.GT.is_het())
p = hl.plot.histogram(hets_mt.AB, range=(0, 1))
p.xaxis.axis_label = 'Allelic balance'
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 183:==================================================>  (114 + 6) / 120]

## Apply filters

Remove all variants failing any of the VQSR filters.

**NOTE:** We must not used the split dataset for filtering VQSR. This is because VQSR consideres all alleles in the site jointly (unless [allele specific annotations](https://gatk.broadinstitute.org/hc/en-us/articles/360035890551?id=9622) have been used).

**FIXME:** be smart about when I use `hl.variant_qc()`.

In [301]:
mt = mt.filter_rows(mt.filters == '')

Now that we have done VQSR filtering, we may split the multi-allelic sites in the main dataset.

In [6]:
# NOTE: permit_shuffle is needed when there are duplicate loci, one of which has multiple alternate alleles. Apparently, this dataset has such sites.
mt = hl.split_multi_hts(mt, permit_shuffle=True)

Filter genotypes with low quality (GQ), using GQ > 20 for SNPs and GQ > 40 for indels.

After the filter some sites may have become invariant. These are removed.

NOTE: Many (or most) of the lower quality genotypes, i.e. the low-end tail of the histogram above, are indels.

In [302]:
mt = mt.filter_entries(hl.if_else(
    hl.is_snp(mt.alleles[0],mt.alleles[1]),
    mt.GQ > 20,
    mt.GQ > 40))

Filter all heterozygotes with allelic balance outside the range of $]0.25;0.75[$.

In [303]:
mt = mt.filter_entries(hl.if_else(
    mt.GT.is_het(),
    (mt.AB > 0.25) & (mt.AB < 0.75),
    True))

Remove variants that significantly deviate from HWE. We use a p-value of $10^{-9}$ for SNPs and $10^{-6}$ for indels. Since our criteria for indels is 1000 times more stringent than for SNPs, we cannot expect the indels to carry information about population structure and inbreeding.

In [304]:
# Update variant statistics.
mt = hl.variant_qc(mt)

mt = mt.filter_rows(hl.if_else(
    hl.is_snp(mt.alleles[0], mt.alleles[1]),
    mt.variant_qc.p_value_hwe > 1e-9,
    mt.variant_qc.p_value_hwe > 1e-6))

Remove singletons.

In [305]:
mt = mt.filter_rows(mt.variant_qc.AC[1] > 1)

In [306]:
variant_counts(mt)

[Stage 184:====================================================>(119 + 1) / 120]

,
n_variants,142496.000000
snps_fraction,0.878663
insertions_fraction,0.055840
deletions_fraction,0.065497
mnp_fraction,0.000000
complex_fraction,0.000000
star_fraction,0.000000


## Sample QC

Below we see histograms of sample mean genotype quality and genotype depth. Most samples seem to have good depth and quality, although the deviation between samples is quite large. There are some samples with low depth and quality, but we will not worry about these.

In [307]:
mt = hl.sample_qc(mt)

In [308]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(0,100), legend='Mean Sample GQ')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 185:===================================================> (116 + 4) / 120]

In [309]:
p = hl.plot.histogram(mt.sample_qc.dp_stats.mean, range=(0,60), legend='Mean Sample DP')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 188:====================================================>(119 + 1) / 120]

Below is a histogram of the heterozygote/homozygote ratio.

In [310]:
p = hl.plot.histogram(mt.sample_qc.r_het_hom_var, legend='Het/hom rate')
p.plot_width = 800
p.plot_height = 500
show(p)

[Stage 194:===================================================> (117 + 3) / 120]

Looks like there are a few samples with a lot higher het/hom rate than the rest of the samples. Let's check whether this is due to poor coverage.

In [311]:
het_hom_thres = 3
mt = mt.annotate_cols(high_hom_het=mt.sample_qc.r_het_hom_var > het_hom_thres)

In [312]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.r_het_hom_var, xlabel='DP mean', ylabel='het/hom rate',
                    hover_fields={'Sample': mt.s}, label=mt.high_hom_het)
p.plot_width = 600
p.plot_height = 600
show(p)

[Stage 197:====================================================>(119 + 1) / 120]

The plot above indicates these samples have similar coverage as the other samples, so that doesn't explain the high het/hom rate.

We can also check the MultiQC reports of these samples (file paths below), and we see that they are all of reasonable quality.

In [313]:
high_hethom_samples = mt.filter_cols(mt.sample_qc.r_het_hom_var > het_hom_thres).s.collect()

for sample in high_hethom_samples:
    print('/data/projects/fargen_phase_1/data/single_sample_data/{sample}/multiqc/multiqc_report.html'.format(sample=sample))

[Stage 200:===================================================> (116 + 4) / 120]

/data/projects/fargen_phase_1/data/single_sample_data/FN000909/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN000940/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN001018/multiqc/multiqc_report.html
/data/projects/fargen_phase_1/data/single_sample_data/FN001019/multiqc/multiqc_report.html


In the genealogy summary file (`/fargen/fargen_phase_1_utils/multi_sample/joint_genotyping/metadata/genealogy/individuals_summary.csv`), it seems that all these samples have reasonably deep roots in the Faroes. This means we have no reason to suspect this difference is due to these samples being from different populations.

These four samples most likely have high het/hom rate due to poor data quality. One potential reason for this is contamination of the sample in the lab.

We will **discard high het/hom rate samples**, as they may skew further analyses down the line.

In [314]:
mt = mt.filter_cols(mt.sample_qc.r_het_hom_var < het_hom_thres)

Remove sites that may have become invariant after removing these samples.

In [315]:
mt = mt.filter_rows(mt.variant_qc.AC[1] > 0)

In [316]:
n_variants, n_samples = mt.count()
print('Number of variants: ' + str(n_variants))
print('Number of samples: ' + str(n_samples))

[Stage 204:==================================================>  (115 + 5) / 120]

Number of variants: 142496
Number of samples: 469


[Stage 204:===================================================> (116 + 4) / 120]

## Variant counts

In [317]:
variant_counts(mt)

[Stage 205:====================================================>(119 + 1) / 120]

,
n_variants,142496.000000
snps_fraction,0.878663
insertions_fraction,0.055840
deletions_fraction,0.065497
mnp_fraction,0.000000
complex_fraction,0.000000
star_fraction,0.000000


## Write data to file

In [318]:
if False:
    mt.write(BASE_DIR + '/data/mt/high_quality_variants.mt', overwrite=True)

[Stage 208:==============================================>        (51 + 9) / 60]2021-09-30 13:52:13 Hail: INFO: Ordering unsorted dataset with network shuffle
[Stage 211:======================================================>(59 + 1) / 60]2021-09-30 13:54:34 Hail: INFO: wrote matrix table with 142496 rows and 469 columns in 60 partitions to /home/olavur/experiments/2020-11-13_fargen1_exome_analysis/data/mt/high_quality_variants.mt
    Total size: 786.13 MiB
    * Rows/entries: 786.08 MiB
    * Columns: 49.88 KiB
    * Globals: 11.00 B
    * Smallest partition: 619 rows (2.73 MiB)
    * Largest partition:  3085 rows (17.47 MiB)


## Summary

In this notebook we have:

* Split multi-allelic sites
* Filtered variants failing VQSR filter
* Filtered variants with genotype quality (GQ)
    * GQ > 20 for SNPs
    * GQ > 40 for indels
* Filtered variants with allelic balance outside the range $[0.25, 0.75]$
* Filtered variants failing HWE filter
    * $p > 10^{-9}$ for SNPs
    * $p > 10^{-6}$ for indels
* Removed singletons
* Removed four samples with abnormally high het/hom rate (>3)
* Filtered variants with allele count equal to zero